### Importando a biblioteca

In [ ]:
###!pip install --upgrade scikit-learn==0.20.3
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

%load_ext google.colab.data_table
#pd.set_option ('display.max_rows', 99999)

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


### Carregando o conjunto de dados
O conjunto de dados no arquivo CSV.
O conjunto de dados utilizado do Kaggle sobre Risco de crédito alemão:
https://www.kaggle.com/datasets/uciml/german-credit

## Descrição das colunas (váriaveis/atributos/features) do conjunto de dados


```
Age (numeric)
Sex (text: male, female)
Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
Housing (text: own, rent, or free)
Saving accounts (text - little, moderate, quite rich, rich)
Checking account (text - little, moderate, rich, )
Credit amount (numeric, in DM)
Duration (numeric, in month)
Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)
Risk (text: good, bad)

```




In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/digiampietri/basesDeDados/main/german-credit.csv")

In [ ]:
for columns in ['Age','Sex','Job','Housing','Saving accounts','Checking account','Credit amount','Duration','Purpose']:
  print (dados[columns].value_counts())

27    51
26    50
23    48
24    44
28    43
25    41
30    40
35    40
36    39
31    38
29    37
32    34
33    33
34    32
37    29
22    27
40    25
38    24
42    22
39    21
46    18
43    17
47    17
44    17
41    17
45    15
20    14
21    14
49    14
50    12
48    12
54    10
57     9
52     9
51     8
55     8
63     8
61     7
53     7
60     6
65     5
58     5
64     5
66     5
74     4
67     3
68     3
56     3
59     3
75     2
19     2
62     2
70     1
Name: Age, dtype: int64
male      690
female    310
Name: Sex, dtype: int64
2    630
1    200
3    148
0     22
Name: Job, dtype: int64
own     713
rent    179
free    108
Name: Housing, dtype: int64
little        603
moderate      103
quite rich     63
rich           48
Name: Saving accounts, dtype: int64
little      274
moderate    269
rich         63
Name: Checking account, dtype: int64
1478    3
1262    3
1258    3
1275    3
1393    3
       ..
1459    1
882     1
3758    1
1136    1
4576    1
Name: Credit amount,

In [ ]:
dados.head()

,Id,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


In [ ]:
dados[:500]

,Id,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...,...
495,495,20,male,2,own,NaN,little,2996,24,furniture/equipment,bad
496,496,29,male,3,rent,moderate,moderate,9034,36,furniture/equipment,bad
497,497,40,male,2,own,little,NaN,1585,24,furniture/equipment,good
498,498,32,male,1,own,little,moderate,1301,18,radio/TV,good


Acredito que este precise excluir algumas linhas, já que alguns atributos tem menos linhas que outros

In [ ]:
dados.groupby('Risk').count()

,Id,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
Risk,,,,,,,,,,
bad,300,300,300,300,300,268,254,300,300,300
good,700,700,700,700,700,549,352,700,700,700


In [ ]:
dados.info


<bound method DataFrame.info of       Id  Age     Sex  Job Housing Saving accounts Checking account  \
0      0   67    male    2     own             NaN           little   
1      1   22  female    2     own          little         moderate   
2      2   49    male    1     own          little              NaN   
3      3   45    male    2    free          little           little   
4      4   53    male    2    free          little           little   
..   ...  ...     ...  ...     ...             ...              ...   
995  995   31  female    1     own          little              NaN   
996  996   40    male    3     own          little           little   
997  997   38    male    2     own          little              NaN   
998  998   23    male    2    free          little           little   
999  999   27    male    2     own        moderate         moderate   

     Credit amount  Duration              Purpose  Risk  
0             1169         6             radio/TV  good  

##Convertendo as características para características numéricas

In [ ]:
#Sex: [male, female]
dados['Sex'] = np.where(dados['Sex'] == "male", 0, dados['Sex'])
dados['Sex'] = np.where(dados['Sex'] == "female", 1, dados['Sex'])

#Housing [own, rent, free]
dados['Housing'] = np.where(dados['Housing'] == "own", 1, dados['Housing'])
dados['Housing'] = np.where(dados['Housing'] == "rent", 2, dados['Housing'])
dados['Housing'] = np.where(dados['Housing'] == "free", 3, dados['Housing'])

#Saving Accounts [little, moderate, quite rich, rich]
dados['Saving accounts'] = dados['Saving accounts'].fillna(0)
dados['Saving accounts'] = np.where(dados['Saving accounts'] == "little", 1, dados['Saving accounts'])
dados['Saving accounts'] = np.where(dados['Saving accounts'] == "moderate", 2, dados['Saving accounts'])
dados['Saving accounts'] = np.where(dados['Saving accounts'] == "quite rich", 3, dados['Saving accounts'])
dados['Saving accounts'] = np.where(dados['Saving accounts'] == "rich", 4, dados['Saving accounts'])

#Checking account [little, moderate, rich]
dados['Checking account'] = dados['Checking account'].fillna(0)
dados['Checking account'] = np.where(dados['Checking account'] == "little", 1, dados['Checking account'])
dados['Checking account'] = np.where(dados['Checking account'] == "moderate", 2, dados['Checking account'])
dados['Checking account'] = np.where(dados['Checking account'] == "rich", 3, dados['Checking account'])

#Purpose: [car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others]
dados['Purpose'] = np.where(dados['Purpose'] == "car", 1, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "furniture/equipment", 2, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "radio/TV", 3, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "domestic appliances", 4, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "repairs", 5, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "education", 6, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "business", 7, dados['Purpose'])
dados['Purpose'] = np.where(dados['Purpose'] == "vacation/others", 8, dados['Purpose'])

#Risk [bad, good]
dados['Risk'] = np.where(dados['Risk'] == "bad", 0, dados['Risk'])
dados['Risk'] = np.where(dados['Risk'] == "good", 1, dados['Risk'])

dados.head()

,Id,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,0,2,1,0,1,1169,6,3,1
1,1,22,1,2,1,1,2,5951,48,3,0
2,2,49,0,1,1,1,0,2096,12,6,1
3,3,45,0,2,3,1,1,7882,42,2,1
4,4,53,0,2,3,1,1,4870,24,1,0


In [ ]:
dados.groupby('Risk').count()

,Id,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
Risk,,,,,,,,,,
0,300,300,300,300,300,300,300,300,300,300
1,700,700,700,700,700,700,700,700,700,700


##Seleção de características

In [ ]:
feature_cols = ['Age','Sex','Job','Housing','Saving accounts','Checking account','Credit amount','Duration','Purpose']
X = dados[feature_cols]
Y = dados.Risk

In [ ]:
Y
Y=Y.astype('int')

##Separação de Conjunto de dados: Treinamento e teste

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1) # 80% para treinamento (será divido em dois) e 20% para teste
x_train1, x_train2, y_train1, y_train2 = train_test_split(x_train, y_train, test_size=0.5, shuffle=False)

In [ ]:
print('Tamanho do conjunto de treinamento: %i' %x_train.shape[0])
print('Tamanho do conjunto de treinamento1: %i' %x_train1.shape[0])
print('Tamanho do conjunto de treinamento2: %i' %x_train2.shape[0])
print('Tamanho do conjunto de teste: %i' %y_test.shape[0])

Tamanho do conjunto de treinamento: 800
Tamanho do conjunto de treinamento1: 400
Tamanho do conjunto de treinamento2: 400
Tamanho do conjunto de teste: 200


##Treinando os modelos explicáveis

Iremos construir dois modelos explicáveis inicialmente:

1a) Árvore de Decisão para classificar o conjunto de testes (treinada com o conjunto de treinamento 1);

2a) Árvore de Decisão para classificar o conjunto de testes (treinada com o conjunto de treinamento 2);

In [ ]:

classificadorExplicavel1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)
classificadorExplicavel2 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Treinando os quatro modelos iniciais
# Usando o conjunto de treinamento e a respectiva variável alvo
modeloExplicavelOriginal = classificadorExplicavel1.fit(x_train1,y_train1)

# Usando o segundo conjunto de treinamento e a respectiva variável alvo
modeloExplicavel2 = classificadorExplicavel2.fit(x_train2,y_train2)


predicaoExplicavel = modeloExplicavelOriginal.predict(x_test)

predicaoExplicavel2 = modeloExplicavel2.predict(x_test)


## Construindo (ou Treinando) os Modelos (parte 2)

Iremos instanciar e treinar os modelos caixa-preta a serem utilizados no laço.

Cada modelo caixa-preta é treinado uma única vez.

**Caixa-preta** (treinado com o conjunto de treinamento 1) para classificar o conjunto de testes, mas também para classificar o conjunto de treinamento 2 (para produzir dados a serem usados pelo modelo explicável [para explicar o caixa-preta]);

**Criamos um arranjo de modelos** para ser utilizado em um laço nos testes que seguem.

In [ ]:
from sklearn.dummy import DummyClassifier # Este classificador será usado como um baseline

# Modelos que poderão ser usados como "caixa-preta"
from sklearn.ensemble import RandomForestClassifier # Floresta Aleatória
from sklearn.linear_model import LogisticRegression # Regressão Logística
from sklearn.neural_network import MLPClassifier # Rede Neural MLP
from sklearn.naive_bayes import GaussianNB # Classificador Baysiano que utiliza distribuição Gaussiana
from sklearn.neighbors import KNeighborsClassifier # KNN - K-Vizinhos mais próximos
from sklearn import svm # SVM: Máquina de Vetores de Suporte


### Estes classificadores serão tratados como 'caixa-preta'
classificadorCaixaPreta1 = DummyClassifier(strategy='most_frequent', random_state=42, constant = None)
classificadorCaixaPreta2 = DecisionTreeClassifier()
classificadorCaixaPreta3 = RandomForestClassifier()
classificadorCaixaPreta4 = svm.SVC()
classificadorCaixaPreta5 = svm.SVC(kernel='poly')
classificadorCaixaPreta6 = LogisticRegression(max_iter=5000)
classificadorCaixaPreta7 = MLPClassifier(max_iter=5000)
classificadorCaixaPreta8 = GaussianNB()
classificadorCaixaPreta9 = KNeighborsClassifier(n_neighbors = 10)

# Criando um arranjo com o nome dos modelos (para aparecer impresso nos resultados)
nomes = ["Dummy", "Árvore de Decisão", "Floresta Aleatoria", "SVM (linear)", "SVM (polinomial)", "Regressão Logística", "Multilayer perceptron", "Gaussian Naive Bayes", "KNN"]


# Criando modelos para os classifidores caixa-preta
# Usando o conjunto de treinamento 1 e a respectiva variável alvo
modeloCaixaPreta1 = classificadorCaixaPreta1.fit(x_train1,y_train1)
modeloCaixaPreta2 = classificadorCaixaPreta2.fit(x_train1,y_train1)
modeloCaixaPreta3 = classificadorCaixaPreta3.fit(x_train1,y_train1)
modeloCaixaPreta4 = classificadorCaixaPreta4.fit(x_train1,y_train1)
modeloCaixaPreta5 = classificadorCaixaPreta5.fit(x_train1,y_train1)
modeloCaixaPreta6 = classificadorCaixaPreta6.fit(x_train1,y_train1)
modeloCaixaPreta7 = classificadorCaixaPreta7.fit(x_train1,y_train1)
modeloCaixaPreta8 = classificadorCaixaPreta8.fit(x_train1,y_train1)
modeloCaixaPreta9 = classificadorCaixaPreta9.fit(x_train1,y_train1)


# Criando um arranjo dos modelos caixa-preta criados
modelosCaixaPreta = [modeloCaixaPreta1, modeloCaixaPreta2, modeloCaixaPreta3, modeloCaixaPreta4, modeloCaixaPreta5, modeloCaixaPreta6, modeloCaixaPreta7, modeloCaixaPreta8, modeloCaixaPreta9]

## Avaliação dos Modelos
Os modelos são avaliados em relação à acurácia, precisão, revocação e medida F1.

Os itens **a** e **b** são relacionados unicamente com o modelo explicável e por isso são executados à parte.

**O foco principal é analisar:**

**[a] Desempenho do modelo 'explicável' 1 (treinado usando o conjunto de treinamento 1);**


[b] Desempenho do modelo 'explicável' 2 (treinado usando o conjunto de treinamento 2);

**[1] Desempenho do modelo 'caixa-preta';**

[2] Desempenho do modelo 'explicável' 1 "explicando" o modelo 'caixa-preta';

[3] Desempenho do modelo 'explicável' 2 (treinamento com variável alvo original) "explicando" o modelo 'caixa-preta';

**[4] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta) "explicando" o modelo 'caixa-preta;**

**[5] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta), mas aplicado à variável alvo do conjunto de testes.**


In [ ]:
print('[a] Resultados para Modelo Explicável (treinado com o conjunto de treinamento 1)')
print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicavel))
report=metrics.classification_report(y_test,predicaoExplicavel)
print(report)
print(confusion_matrix(y_test, predicaoExplicavel))

print('\n[b] Resultados para Modelo Explicável 2 (treinado com o conjunto de treinamento 2)')
print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicavel2))
report=metrics.classification_report(y_test,predicaoExplicavel2)
print(report)
print(confusion_matrix(y_test, predicaoExplicavel2))

[a] Resultados para Modelo Explicável (treinado com o conjunto de treinamento 1)
Acurácia: 0.685
              precision    recall  f1-score   support

           0       0.45      0.29      0.35        59
           1       0.74      0.85      0.79       141

    accuracy                           0.69       200
   macro avg       0.59      0.57      0.57       200
weighted avg       0.65      0.69      0.66       200

[[ 17  42]
 [ 21 120]]

[b] Resultados para Modelo Explicável 2 (treinado com o conjunto de treinamento 2)
Acurácia: 0.685
              precision    recall  f1-score   support

           0       0.46      0.41      0.43        59
           1       0.76      0.80      0.78       141

    accuracy                           0.69       200
   macro avg       0.61      0.60      0.61       200
weighted avg       0.67      0.69      0.68       200

[[ 24  35]
 [ 28 113]]


## Construindo (ou Treinando) os Modelos (parte 3)

Dentro do laço um novo modelo é treinado:

**Árvore de Decisão** para explicar o modelo caixa-preta(treinada com o conjunto de treinamento 2, mas usando como alvo a classificação produzida pelo modelo caixa-preta).

Este modelo também será testado para explicar a saída da Floresta Aleatória, mas também para classificar a variável alvo do conjunto de testes.

## Avaliação dos Modelos
Os modelos são avaliados em relação à acurácia, precisão, revocação e medida F1.

** Neste bloco serão executados os testes que envolvem os modelos caixa-preta. ** Estão destacados os principais resultados para a pesquisa, porém os cinco resultados serão produzidos.


**[1] Desempenho do modelo 'caixa-preta';**

[2] Desempenho do modelo 'explicável' 1 "explicando" o modelo 'caixa-preta';

[3] Desempenho do modelo 'explicável' 2 (treinamento com variável alvo original) "explicando" o modelo 'caixa-preta';

**[4] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta) "explicando" o modelo 'caixa-preta;**

**[5] Desempenho do modelo 'explicável' 3 (treinamento com saída do caixa-preta), mas aplicado à variável alvo do conjunto de testes.**

In [ ]:
cont = 0
for m in modelosCaixaPreta:

  print("\n############################ " + nomes[cont] + " ############################\n")
  cont = cont + 1
  classificadorExplicavel3 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

  # Realizando a predição inicial (caixa-preta para o teste)
  predicaoCaixaPreta = m.predict(x_test)
  # Produzindo as predições da variável alvo que serão usadas para explicar o caixa-preta
  predicaoCaixaPreta2a = m.predict(x_train2)

  # Treinando o modelo explicativo, com base na predição do modelo caixa-preta
  modeloExplicandoCaixaPreta = classificadorExplicavel3.fit(x_train2,predicaoCaixaPreta2a)

  # Realizando as predições com base no que foi aprendido com o modelo caixa-preta
  predicaoExplicandoCaixaPreta = modeloExplicandoCaixaPreta.predict(x_test)

  print('\n[1] Resultados para o Modelo Caixa-Preta')
  print("Acurácia:",metrics.accuracy_score(y_test, predicaoCaixaPreta))
  report=metrics.classification_report(y_test,predicaoCaixaPreta)
  print(report)
  print(confusion_matrix(y_test, predicaoCaixaPreta))


  print('\n[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicavel))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicavel)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicavel))

  print('\n[3] Resultados para o Modelo Explicável 2 (treinado com os dados de treinamento 2, mas comparando saída com a do caixa preta e não com o target original ["explicando" o caixa-preta])')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicavel2))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicavel2)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicavel2))

  print('\n[4] Resultados para o Modelo Explicável explicando o Caixa-Preta (este é o único modelo treinado com esta finalidade)')
  print("Acurácia:",metrics.accuracy_score(predicaoCaixaPreta, predicaoExplicandoCaixaPreta))
  report=metrics.classification_report(predicaoCaixaPreta,predicaoExplicandoCaixaPreta)
  print(report)
  print(confusion_matrix(predicaoCaixaPreta, predicaoExplicandoCaixaPreta))

  print('\n[5] Resultados do Modelo Explicável treinado com a saída do Caixa-Preta e aplicado aos dados de teste')
  print("Acurácia:",metrics.accuracy_score(y_test, predicaoExplicandoCaixaPreta))
  report=metrics.classification_report(y_test,predicaoExplicandoCaixaPreta)
  print(report)
  print(confusion_matrix(y_test, predicaoExplicandoCaixaPreta))

  print("\n\n\n")


############################ Dummy ############################


[1] Resultados para o Modelo Caixa-Preta
Acurácia: 0.705
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.70      1.00      0.83       141

    accuracy                           0.70       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.50      0.70      0.58       200

[[  0  59]
 [  0 141]]

[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)
Acurácia: 0.81
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.81      0.90       200

    accuracy                           0.81       200
   macro avg       0.50      0.41      0.45       200
weighted avg       1.00      0.81      0.90       200

[[  0   0]
 [ 38 162]]

[3] Resultados para 

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classificat

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

[[200]]

[5] Resultados do Modelo Explicável treinado com a saída do Caixa-Preta e aplicado aos dados de teste
Acurácia: 0.705
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.70      1.00      0.83       141

    accuracy                           0.70       200
   macro avg       0.35      0.50      0.41       200
weighted avg       0.50      0.70      0.58       200

[[  0  59]
 [  0 141]]





############################ Árvore de Decisão ############################


[1] Resultados para o Modelo Caixa-Preta
Acurácia: 0.7


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.49      0.54      0.52        59
           1       0.80      0.77      0.78       141

    accuracy                           0.70       200
   macro avg       0.65      0.65      0.65       200
weighted avg       0.71      0.70      0.70       200

[[ 32  27]
 [ 33 108]]

[2] Resultados para o Modelo Explicável 1 - comparando saída com a do caixa preta e não com o target original ("explicando" o caixa-preta)
Acurácia: 0.705
              precision    recall  f1-score   support

           0       0.58      0.34      0.43        65
           1       0.73      0.88      0.80       135

    accuracy                           0.70       200
   macro avg       0.66      0.61      0.61       200
weighted avg       0.68      0.70      0.68       200

[[ 22  43]
 [ 16 119]]

[3] Resultados para o Modelo Explicável 2 (treinado com os dados de treinamento 2, mas comparando saída com a do caixa preta e não com o target